In [ ]:
# Importing the pandas module
import pandas as pd

In [ ]:
# Importing the numpy module
import numpy as np

In [ ]:
# Import os module
import os


In [ ]:
# The base directory
base_directory = os.getcwd()

In [ ]:
# Create a root directory (necessary since we would have the code in its own folder)
root_directory = os.path.abspath(os.path.join(base_directory, ".."))

In [ ]:
# this is the path to the templates folder holding the excel templates
templates_path = os.path.join(root_directory,"data","Templates")


In [ ]:
# This is the path to the historic_data_folder holding the legacy csv and legacy resume folder
historic_data_path = os.path.join(root_directory,"data","Historic_data")

In [ ]:
# This is the path to the resume folder
resume_path = os.path.join(historic_data_path,"Resumes")

In [ ]:
# This is the path to the folder holding excel files that we are writing to
transformed_file_path = os.path.join(root_directory,"transformed_data")

In [ ]:
# Constructs to the full path to the legacy csv
legacy_csv_file_path = os.path.join(historic_data_path,"candidates.presence.latest.csv")

In [ ]:
# Create the historical dataframe from the legacy csv path
historical_data_df = pd.read_csv(legacy_csv_file_path)

In [ ]:
# Normalize column names to remove leading or trailing spaces
historical_data_df.columns = historical_data_df.columns.str.strip()

In [ ]:
# File path to Candidates Template excel file
candidates_template_file_path = os.path.join(templates_path,"Candidates Template.xlsx")

In [ ]:
# File path to Application Template Excel File
applications_template_file_path = os.path.join(templates_path,"Applications Template.xlsx")

In [ ]:
# File path to Jobs Template Excel File
jobs_template_file_path = os.path.join(templates_path,"Jobs Template.xlsx")

In [ ]:
# File path to Attachments Template Excel File
attachments_template_file_path = os.path.join(templates_path,"Attachments Template.xlsx")

In [ ]:
# Quick look at the column names
historical_data_df.columns

In [ ]:
historical_data_df.dtypes

In [ ]:
# List of columns to be dropped
dropped_columns = ["Company", "Schools", "Zip", "Snoozed Until", "Requisition For Hire ID", "Requisition For Hire Requisition Code", "Profile Archive Reason", "Start Date"]

In [ ]:
# Drops the columns in our dropped_columns list
def drop_columns(source_df, column_list):
    for column in column_list:
        if column in source_df.columns:
            source_df = source_df.drop(columns=[column])
            
    return source_df

In [ ]:
# List of columns to convert to nullable string
nullable_string_columns = ["Email", "Phone", "Address", "City", "State", "Links", "Files"]

In [ ]:
# Converts column datatypes to nullable string datatype
def convert_to_nullable_string(source_df, column_list):
    for column in column_list:
        if column in source_df.columns:
            source_df[column] = source_df[column].astype("string")
    return source_df

In [ ]:
# List of columns to convert to datetime64
datetime64_columns = ["Created At", "Archived At", "Hired"]

In [ ]:
# Converts column datatypes to datetime64 datatype
def convert_to_datetime64(source_df, column_list):
    for column in column_list:
        if column in source_df.columns:
            source_df[column] = pd.to_datetime(source_df[column], errors='coerce')
    return source_df

In [ ]:
historical_data_df = drop_columns(historical_data_df, dropped_columns)

In [ ]:
historical_data_df = convert_to_nullable_string(historical_data_df, nullable_string_columns)

In [ ]:
historical_data_df = convert_to_datetime64(historical_data_df, datetime64_columns)

In [ ]:
# Final Schema
historical_data_df.dtypes

In [ ]:
# Returns the full address            
def get_address(row):
    address_parts = []
    if pd.notnull(row["Address"]):
        address_parts.append(row["Address"])
    if pd.notnull(row["City"]):
        address_parts.append(row["City"])
    if pd.notnull(row["State"]):
        address_parts.append(row["State"])
    return ", ".join(address_parts) if address_parts else np.nan
    

In [ ]:
historical_data_df["Address"] = historical_data_df.apply(get_address, axis=1)

In [ ]:
# Drop the "State" and "City" columns
historical_data_df = historical_data_df.drop(columns=["State", "City"])

In [ ]:
# Strip whitespace from all nullable "string" columns
historical_data_df = historical_data_df.apply(lambda col: col.str.strip() if pd.api.types.is_string_dtype(col) else col)


In [ ]:
# Strip whitespace from all string columns
historical_data_df = historical_data_df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

In [ ]:
# Transform Candidates File
def transform_candidates(source_df, template_file_path):
    
    # Create a list of the necessary columns
    required_columns = ["Contact ID", "Candidate Name", "Email", "Phone", "Address", "Links"]
    candidates_df = source_df[required_columns].copy()
    
    # split Candidate Name into two separate columns for first name and last name
    candidates_df[["First name", "Last name"]] = candidates_df["Candidate Name"].str.split(" ", n=1, expand=True)
    
    # Clean up the "Phone" column to only have digits as values, e.g. 6137892379
    candidates_df["Phone"] = candidates_df["Phone"].astype("string").str.replace(r"\D", "", regex=True)
    
    # Rename columns according to the standard provided in the template
    candidates_df = candidates_df.rename(columns=
                        {"Contact ID" : "Candidate ID", 
                         "Links":"Website"})
    
    # Ensure correct ordering of columns
    template_cols = pd.read_excel(template_file_path,index_col=0).columns.tolist()
    final_candidates_df = candidates_df[template_cols]
    
    return final_candidates_df

In [ ]:
# Helper function to get the values in the "Status" column we will be creating.
# If the "Hired" column is not null, it means the applicant was hired.

def derive_status(row):
    
    if pd.notnull(row["Hired"]):
        return "Hired"
    else:
        return "Rejected"

In [ ]:
# Transform Applications File
def transform_applications(source_df, template_file_path):
    
    
    # Create a list of required columns
    required_columns = ["App ID", "Contact ID", "Hired", "Archive Reason", "Posting Title", "Sources", "Current Stage", "Created At (GMT)", "Archived At (GMT)"]
    applications_df = source_df[required_columns].copy()
    
    # Derive the "Status" column from the "Hired" column
    applications_df["Status"] = applications_df.apply(derive_status, axis=1)
    
    # Rename columns to match template standards
    applications_df = applications_df.rename(columns={"App ID":"Application ID",
                                                      "Contact ID":"Candidate ID",
                                                      "Archive Reason":"Rejection Reason", 
                                                      "Posting Title":"Job Name",
                                                      "Sources":"Source",
                                                      "Current Stage":"Stage",
                                                      "Created At (GMT)":"Application Date",
                                                      "Archived At (GMT)":"Rejection Date",
                                                      "Hired":"Hire Date"})
    
    # We want null the values in the "Rejection Reason" column that are "Hired" so we can set them to NaN
    applications_df.loc[applications_df["Rejection Reason"] == "Hired", "Rejection Reason"] = np.nan
    
    # Ensure correct ordering of columns
    template_cols = pd.read_excel(template_file_path,index_col=0).columns.tolist()
    final_applications_df = applications_df[template_cols]

    return final_applications_df
    

In [ ]:
# Transform Jobs File
def transform_jobs(source_df, template_file_path):
    
    # Create the list of required columns
    required_columns = ["Posting Title", "Posting Team", "Posting Level"]
    jobs_df = source_df[required_columns].copy()
    
    # Rename colums to match template standards
    jobs_df = jobs_df.rename(columns={"Posting Title":"Job Name",
                                      "Posting Team":"Department",
                                      "Posting Level":"Office"})
    
    # Add optional columns with NaN values in case future data includes them
    jobs_df["Open Date"] = np.nan
    jobs_df["Closed Date"] = np.nan
    jobs_df["Description"] = np.nan
    
    # Ensure correct ordering of columns
    template_cols = pd.read_excel(template_file_path,index_col=0).columns.tolist()
    final_jobs_df = jobs_df[template_cols]
    
    return final_jobs_df
    

In [ ]:
# Creates a list of resume file paths from a folder path containing the candidate's resume files.
def get_resume_list(folder_path):
    
    resume_list = [os.path.join(folder_path, resume) for resume in os.listdir(folder_path) if resume.lower().endswith(".pdf") 
                   and os.path.isfile(os.path.join(folder_path, resume))]
    return resume_list


In [ ]:
# Checks if the values in the "File" column are in our list of resumes.
# If they are then the file provided is a resume.
# If no file was provided we return null.
def file_type(row, resume_list):
    
    # we check all the rows in our dataframe for the condition
    if pd.notnull(row["Files"]):
        file = str(row["Files"]).lower()
        for path in resume_list:
            file_name = os.path.basename(path).lower()
            if file == file_name:
                return "Resume"
    return np.nan

In [ ]:
# Checks if the values in the "File" column are in our list of resumes.
# If they are, then we want to return to the full file paths.
# If no file was provided, we return null.
def file_path(row, resume_list):
    if pd.notnull(row["Files"]):
        file = str(row["Files"]).lower().strip()
        for path in resume_list:
            file_name = os.path.basename(path).lower().strip()
            if file == file_name:
                return path
    return np.nan
                

In [ ]:
# Transform Attachments File
def transform_attachments(source_df, resume_list, template_file_path):
    
    # Create list of required columns
    required_columns = ["Contact ID", "Files"]
    attachments_df = source_df[required_columns].copy()
    

    # Check if the file is in our list of resume file paths
    attachments_df["File path"] = attachments_df.apply(lambda row: file_path(row, resume_list), axis=1)
    attachments_df["Files"] = attachments_df.apply(lambda row: file_type(row, resume_list), axis=1)
    
    # Rename columns to comply with template standards
    # I opted to use Contact ID -> Candidate ID since the template allows for choice between Candidate ID and Application ID
    attachments_df = attachments_df.rename(columns={"Contact ID":"Application ID or Candidate ID", 
                                                    "Files":"Attachment Type"})
    
    # Ensure correct ordering of columns
    template_cols = pd.read_excel(template_file_path,index_col=0).columns.tolist()
    final_attachments_df = attachments_df[template_cols]
    
    return final_attachments_df
    
    

In [ ]:
# The path to the resume folder
resume_folder = resume_path

In [ ]:
# Creates a list holding resume file paths
resume_list = get_resume_list(resume_folder)

In [ ]:
transformed_candidates_df = transform_candidates(historical_data_df, candidates_template_file_path)

In [ ]:
transformed_applications_df = transform_applications(historical_data_df, applications_template_file_path)

In [ ]:
transformed_jobs_df = transform_jobs(historical_data_df, jobs_template_file_path)

In [ ]:
transformed_attachments_df = transform_attachments(historical_data_df, resume_list, attachments_template_file_path)

In [ ]:
transformed_candidates_df

In [ ]:
transformed_applications_df

In [ ]:
transformed_jobs_df

In [ ]:
transformed_attachments_df

In [ ]:
# File path to presentation/transformed layer
candidates_presentation = os.path.join(transformed_file_path,"Candidates Template.xlsx")

In [ ]:
# File path to presentation/transformed layer
applications_presentation = os.path.join(transformed_file_path,"Applications Template.xlsx")

In [ ]:
# File path to presentation/transformed layer
jobs_presentation = os.path.join(transformed_file_path,"Jobs Template.xlsx")

In [ ]:
# File path to presentation/transformed layer
attachments_presentation = os.path.join(transformed_file_path,"Attachments Template.xlsx")

In [ ]:
# Write to excel files
def write_to_excel(transformed_df, file_path):
    try:
        transformed_df.to_excel(file_path, index=False)
        return True
    except Exception as e:
        print(f"Error writing to Excel: {e}")
        return False    

In [ ]:
write_to_excel(transformed_candidates_df, candidates_presentation)

In [ ]:
write_to_excel(transformed_applications_df, applications_presentation)

In [ ]:
write_to_excel(transformed_jobs_df, jobs_presentation)

In [ ]:
write_to_excel(transformed_attachments_df, attachments_presentation)